### Pinecone Basics

In [2]:
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# Pinecone Object 
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))
pc

In [6]:
# List pinecone indexes 
pc.list_indexes()

[]

In [7]:
# Create Index

index_name = "products"
pc.create_index(
    name=index_name,
    dimension=2, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1" # For free Pinecone version this is the only choice.
    ) 
)

{
    "name": "products",
    "metric": "cosine",
    "host": "products-v8rrxbr.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 2,
    "deletion_protection": "disabled",
    "tags": null
}

In [8]:
# List pinecone indexes 
pc.list_indexes()

[
    {
        "name": "products",
        "metric": "cosine",
        "host": "products-v8rrxbr.svc.aped-4627-b74a.pinecone.io",
        "spec": {
            "serverless": {
                "cloud": "aws",
                "region": "us-east-1"
            }
        },
        "status": {
            "ready": true,
            "state": "Ready"
        },
        "vector_type": "dense",
        "dimension": 2,
        "deletion_protection": "disabled",
        "tags": null
    }
]

In [9]:
# Insert data to the vector DB, this needs to be a list of tuples [(), ()]
data = [
    ('product_1', [1.0, 2.0]),
    ('product_2', [1.5, 2.5]),
    ('product_3', [2.0, 3.0]),
]

In [10]:
# Upsert the data
pc.Index(name='products').upsert(data)

{'upserted_count': 3}

In [12]:
# Update one of the vector product_2 
data2 = [
    ('product_2', [1.6,2.8])
]
pc.Index(name='products').upsert(data2)

{'upserted_count': 1}

In [16]:
# Query the vector
result = pc.Index(name='products').query(vector=[2.5,3.0], top_k=2, namespace=None)
result

{'matches': [{'id': 'product_3', 'score': 0.99626863, 'values': []},
             {'id': 'product_2', 'score': 0.987002075, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

In [19]:
# Fetch vectors by ID
result = pc.Index(name='products').query(id='product_2', top_k=1, namespace=None)
result

{'matches': [{'id': 'product_2', 'score': 1.00291598, 'values': []}],
 'namespace': '',
 'usage': {'read_units': 5}}

In [ ]:
# Fetch vectors by IDs but pass mutilple of them at the same time as shown below.
pc.Index(name='products').fetch(ids=['product_1', 'product_2']).vectors

{'product_1': Vector(id='product_1', values=[1.0, 2.0], metadata=None, sparse_values=None),
 'product_2': Vector(id='product_2', values=[1.6, 2.8], metadata=None, sparse_values=None)}

### Delete Vector

In [23]:
pc.Index(name='products').delete(ids=['product_3'])

{}